<img src="../images/RC_Logo_EN_0128px.png">

# Discount curves

In [ ]:
import pyvacon.analytics as analytics
import datetime
import matplotlib.pyplot as plt
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import math
import pandas as pd
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

## Definition of discount curves
A discount curve 
- stores discount factors for different maturities,
- has an interpolation to compute discount factors for arbitrary maturities,
- has an extrapolation to compute discount factors for arbitrary maturities after the latest given maturity,
- has a daycounter to convert dates to year fracions to apply the inter- and extrapolation.

Under the assumption of continuous compounding, the discount factor $df$ for a cashflow at maturity $T$ based on a constant discount rate $r$ is defined as

$$df(T) = e^{-rT}$$

where $T$ is the time to maturity as year fraction.


## Creating discount curves
### Setup the specification

To setup a discount curve, we need the following information:



**Object id**

Nearly every structure in the analytics library has an object id. An object id allows for nice logging and exceptions (which object created the message/error) and is also used as identifier for retrieving objects from containers.

In [ ]:
object_id = "Test_DC"

**Reference date**

A reference date is needed for all objects representing market data. Dates entering analytics objects must be analytics ptimes. 

Most of the functions provided by market data objects also get a calculation/valuation date and some logic is applied if this date is after the reference date. For a discount curve, if the valuation date is after the reference date, the forward discount factor is returned.

The help function provides information about the function mentioned in its arguments (in parentheses). Uncomment the line to see the information about the analytics.ptime function.

In [ ]:
refdate = analytics.ptime(2017,1,1,0,0,0)
#help(analytics.ptime)
#print(refdate.to_string())

**Dates and corresponding discount factors**

We need a list of dates and a list of discount factors. Here, we must use the analytics (C++ swigged) vectors to create the list of dates. We use a helper method which just gets a list of number of days and a reference date and returns the resulting dates from adding the number of days to the reference date. To view the created list of dates, uncomment the 4th and 5th line of the following code.

To calculate the discount factors from a constant rate, we need to provide the constant rate and calculate the discount factors according to the desired compounding frequency.

In [ ]:
days_to_maturity = [1, 180, 365, 720, 3*365, 4*365, 10*365]
dates = converter.createPTimeList(refdate, days_to_maturity)
#help(converter.createPTimeList)
#for d in dates:
#    print(d.to_string())
discount_rate = 0.03
df = analytics.vectorDouble()
for d in days_to_maturity:
    df.append(math.exp(-d/365.0*discount_rate))

**Daycount convention**

Additionally, we need to provide the discount curve with a daycount convention. The different options are provided in the enums module. Here, we apply the ACT365FIXED daycount convention where the number of days between two dates is divided by 365. For other examples please refer to the [Rollconventions, daycounters and schedule generation](daycounter_rollconventions_schedules.ipynb) notebook.

**Interpolation type and extrapolation type**

The interpolation is necessary to compute discount dactors for arbitrary maturities until the last maturity provided; the extrapolation is necessary to compute discount factors for arbitratry maturities after the last given maturity. Here, we use HAGAN_DF as interpolation type and use no extrapolation. For an overview of interpolation and extrapolation types please refer to the *enums* module.

The main advantage of the The HAGAN_DF interpolation, also called the monotone convex method (unameliorated version), is that in contrast to the other methods, it ensures forward rates to be positive and is, hence, the probably most suitable interpolation method for financial problems. The method is described in depth [here](..\literature\HaganWest.pdf).

The following diagrams show the differences between the linear and HAGAN_DF interpolation for a newly created discount curve.

In [ ]:
refdate = analytics.ptime(2017,1,1,0,0,0)
dtm = analytics.vectorInt()

n = 1
while n <= 30/6:
    dtm.append(6*n*365) # adding approximately one year (ignoring leap years)
    n = n+1

dates_new = analytics.vectorPTime()
dates_new = converter.createPTimeList(refdate, dtm)

rates = [-0.0065, 0.0003, 0.0059, 0.0086, 0.0101]

dsc_fac = analytics.vectorDouble()
for d in range(len(dtm)):
        dsc_fac.append(math.exp(-rates[d]*dtm[d]/365))
        
dc_linear = analytics.DiscountCurve('dc_linear', refdate, dates_new, dsc_fac, enums.DayCounter.ACT365_FIXED, enums.InterpolationType.LINEAR, enums.ExtrapolationType.NONE)
dc_hagan = analytics.DiscountCurve('dc_hagan', refdate, dates_new, dsc_fac, enums.DayCounter.ACT365_FIXED, enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE)

dtm_n = []
n = 1
while n <= 360:
    dtm_n.append(n*30) # adding approximately one month
    n = n+1

dates_new = []
dates_new = converter.createPTimeList(refdate, dtm_n)

dc_linear_values = []
dc_hagan_values = []
for i in range(len(dates_new)):
    dc_linear_values.append(dc_linear.value(refdate, dates_new[i]))
    dc_hagan_values.append(dc_hagan.value(refdate, dates_new[i]))

daycounter = 'Act365Fixed'
dc = analytics.DayCounter(daycounter)
yfValues = analytics.vectorDouble(len(dates_new))
dc.yf(yfValues, refdate, dates_new)

plt.plot(yfValues, dc_linear_values, '-', label='Linear Interpolation')
plt.plot(yfValues, dc_hagan_values, '-', label='Hagan Interpolation')
plt.title('Discount Curves')
plt.xlabel('Time to Maturity (Years)')
plt.ylabel('Discount Factors')
plt.legend()

### Setup the discount curve

Our discount curve is given the name *dc* and is defined as an analytics.DiscountCurve object which we provide with the information described before. 

In [ ]:
dc = analytics.DiscountCurve(object_id, refdate,dates, df, enums.DayCounter.ACT365_FIXED, enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE)
#help(analytics.DiscountCurve)

## Compute discount factors

This section is only to manually derive some discount factors from the recently created discount curve.

**a. Maturity in 180 days from the reference date**

The value function returning a discount factor needs two arguments: the calculation date (here we use the current reference date) and the maturity for which the discount factor shall be computed. Hence, in a first step, the maturity in 180 days has to be converted into a date. Subsequently, the discount factor is computed using the value function. Finally, the print function gives us the computed discount factor of the value function.

In [ ]:
maturity = converter.getLTime(180, refdate)
#help(converter.getLTime)
#print(maturity.to_string())
df1 = dc.value(refdate, maturity)
#help(dc.value)
print("DF for T=180 days: ", df1)

**b. Forward discount factor for a maturity in 180 days in 90 days**

If the valuation date given is in the future, the value function returns the forward discount factor. The following example gives us the forward discount factor for a maturity in 180 days in 90 days.

In [ ]:
fwd_df = dc.value(converter.getLTime(90, refdate), maturity)
print("Fwd-DF for T=180 days in 90 days: ", fwd_df)

A check proves that the result equaly the forward discount curve.

In [ ]:
df2 = dc.value(refdate, converter.getLTime(90, refdate))
print("Fwd-DF (manual calculation): ", df1/df2)

**c. Valuation dates before the reference date**

Valuation dates before the curves reference date result in an exception (uncomment the following code to see the exception).

In [ ]:
#dc.value(converter.getLTime(-90, refdate), maturity)

**d. Vector of discount factors** 

There is also a version of the value funcion to compute a whole vector of discount factors which may be more efficient then looping over the method returning one value. For simplification we just use the dates vector from the construction.

In [ ]:
dfs = analytics.vectorDouble()
dc.value(dfs, refdate, dates)
print("Vector of DFs:")
for df in dfs: print(df)

To see the two versions of the value function and the arguments required uncomment the following code.

In [ ]:
#help(analytics.DiscountCurve.value)

## Plotting discount curves

To plot a curve one needs to pass the dates (either as a vector or list of datetime/ptimes or as a list of days from the reference date), where the curve will be evaluated and the reference date.

In addition, one can choose if the discount factors or continously coompounded rates will be plotted. To show the second option, we first create a new figure, otherwise the rate will be plotted in the same figure as the discount factors.

In [ ]:
#help(mkt_plot.curve)
mkt_plot.curve(dc,range(1,10*365,30), refdate)
mkt_plot.plt.figure()
mkt_plot.curve(dc,range(1,10*365,30), refdate, True)

## Exercises

- Create a second discount curve with non-flat rate structure.

- Plot the second discount curve and the first discount curve together in one figure.

- Create a third discount curve exactly as the second but with interpolation LINEARLOG and compare the differences of results using dfferent interpolation schemes (note that finding the correct ensemble of dates and discount factors, forward rates may not be smoothly interpolated with LINEARLOG).

- Write a function computing a simply (continuously) compounded rate for a given discount factor and year fraction.

- Write a function computing an annualy compounded rate for a given discount factor and year fraction.

- Find and modify the plot function from above so that the user can additionally choose which compounding rate is used when plotting the rate.